In [1]:
import cv2
import numpy as np

In [13]:
video = cv2.VideoCapture(r"C:\Users\MTC1COB\Downloads\personal\study\handson-learning\loitering_data\aboda_data\video9.avi")
# video = cv2.VideoCapture(r"C:\Users\MTC1COB\Downloads\personal\study\handson-learning\loitering_data\loitering_detection_output\S1_T1_C_video3.mp4")

if video.isOpened() == False:
    print("Error in opening or reading the file")


#collect first 10 frames to get the background model
frame_arr = []
for fid in range(10):
    video.set(cv2.CAP_PROP_POS_FRAMES, fid)
    ret, frame = video.read()
    frame_arr.append(frame)
    

medianframe = np.median(frame_arr, axis=0).astype(dtype = np.uint8)
# cv2.imshow("medianframe",medianframe)
# cv2.waitKey(0)
# cv2.destroyAllWindows()

graymedianframe = cv2.cvtColor(medianframe, cv2.COLOR_BGR2GRAY)

previous_frame_centroid = (0,0)
static_counter = 0
image_arr = []

while video.isOpened():
    ret, frame = video.read()
    if ret:
        orig_image = frame
        #convert current frame to gray scale
        current_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
        #do Gaussian Filtering to smooth noise
        current_frame = cv2.GaussianBlur(current_frame, (3,3), 1, 0)
        
        
        #take abs diff to get mask of object and subtract the background 
        frame_diff = cv2.absdiff(current_frame, graymedianframe)
        medianblur = cv2.medianBlur(frame_diff, 3)
        ret, mask = cv2.threshold(medianblur, 100, 255, cv2.THRESH_BINARY)
        
        kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (3,3))
        dilate = cv2.dilate(mask, kernel , iterations=4)

        
        #morphological closing operation
        mask = cv2.morphologyEx(dilate, cv2.MORPH_CLOSE, (5,5), iterations=10)
        contours, _ = cv2.findContours(mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
        for cnt in contours:
            x,y,w,h = cv2.boundingRect(cnt)
            area = cv2.contourArea(cnt)
            if area > 600:
#                 cv2.rectangle(orig_image, (x,y), (x+w, y+h), (0,255,0), 3)
                m = cv2.moments(cnt)
                cx = int(m["m10"]/m["m00"])
                cy = int(m["m01"]/m["m00"])
                current_frame_centroid = (cx, cy)
                
                if current_frame_centroid == previous_frame_centroid:
                    static_counter += 1
#                 else:
#                     static_counter = 0
                
                previous_frame_centroid = current_frame_centroid
                
                if static_counter >= 200:
                    cv2.rectangle(orig_image, (x,y), (x+w, y+h), (0,0,255), 3)
                else:
                    cv2.rectangle(orig_image, (x,y), (x+w, y+h), (0,255,0), 3)
                   
        image_arr.append(orig_image)
        cv2.imshow("Frame",orig_image)
        waitkey = cv2.waitKey(20)
        if waitkey == 113 or waitkey == ord('q'):
            break
    else:
        break
video.release()
cv2.destroyAllWindows()

In [14]:
out_video = cv2.VideoWriter("aboda_dataset_output.mp4",cv2.VideoWriter_fourcc(*'mp4v'), 24, (720,480))
for index in range(len(image_arr)):
    out_video.write(image_arr[index])
out_video.release()